# 概要
Azure AI Agent Service は、Azure 上で AI エージェントを構築・運用するための統合プラットフォームです。本サービスは、従来の Assistants API を基盤としつつ、Azure の多様なマネージドサービスと組み合わせることで、エンタープライズ向けの複雑なユースケースにも柔軟に対応します。

本ノートブックでは、Azure AI Agent Service を活用したマルチエージェント構成のサンプルコードを紹介します。このコードは、Azure SDK for Python のサンプルコードに含まれていた [AgentTeam](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects/samples/agents/multiagent) クラスがベースになっています。

# 事前準備

## ライブラリのインストール

In [12]:
!pip install azure-ai-projects==1.0.0b5 \
    python-dotenv==1.0.1 \
    azure-identity==1.19.0 --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## ユーザー定義関数

In [56]:
# ------------------------------------
# Based on code from Microsoft Corporation.
# Copyright (c) Microsoft Corporation.
#
# Modified by Yuki Matayoshi, 2025.
#
# Licensed under the MIT License.
# ------------------------------------

import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional


def fetch_current_datetime(format: Optional[str] = None) -> str:
    """
    現在の時刻を JSON 文字列として取得します。オプションでフォーマットを指定できます。

    :param format (Optional[str]): 現在の時刻を返す形式。デフォルトは None で、標準形式を使用します。
    :return: JSON 形式の現在の時刻。
    :rtype: str
    """
    current_time = datetime.datetime.now()

    if format:
        time_format = format
    else:
        time_format = "%Y-%m-%d %H:%M:%S"

    time_json = json.dumps({"current_time": current_time.strftime(time_format)})
    return time_json


def fetch_weather(location: str) -> str:
    """
    指定された場所の天気情報を取得します。

    :param location (str): 天気情報を取得する場所。
    :return: JSON 文字列としての天気情報。
    :rtype: str
    """
    # ダミーの天気データを使用します。
    dummy_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = dummy_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json


def send_email(recipient: str, subject: str, body: str) -> str:
    """
    指定の件名と本文を含むメールを宛先に送信します。

    :param recipient (str): 宛先のメールアドレス。
    :param subject (str): メールの件名。
    :param body (str): メールの本文
    :return: 確認メッセージ。
    :rtype: str
    """
    # ダミーのメール送信処理を実行します。
    print(f"Sending email to {recipient}...")
    print(f"Subject: {subject}")
    print(f"Body:\n{body}")

    message_json = json.dumps({"message": f"Email successfully sent to {recipient}."})
    return message_json


def convert_temperature(celsius: float) -> str:
    """
    温度を摂氏から華氏に変換します。

    :param celsius (float): 摂氏での温度。
    :rtype: float

    :return: 華氏での温度。
    :rtype: str
    """
    fahrenheit = (celsius * 9 / 5) + 32
    return json.dumps({"fahrenheit": fahrenheit})


## 指示テンプレート

In [57]:
TEAM_LEADER_INSTRUCTIONS = """
あなたの名前は '{agent_name}' です。あなたはエージェントチームのリーダーです。 
あなたのチーム名は '{team_name}' です。 
あなたの役割は、ユーザーからのリクエストを受け取り、チームのメンバーを活用してタスクを完了させることです。 
リクエストを受け取った際に行うべき唯一のことは、どのチームメンバーが次にどのタスクを担当すべきかを評価することです。 
あなたは提供されている `create_task` 関数を使い、適切なエージェントにタスクを割り当てます。 
その際、誰にタスクを割り当てたのか、そしてその理由を説明してください。 
チーム内のすべてのメンバーのスキルを活用することが重要です。 
元のユーザーリクエストが完全に処理されたと判断した場合は、新しいタスクを作成する必要はありません。 
ここに、あなたのチームに所属する他のエージェントを示します：
"""

TEAM_LEADER_INITIAL_REQUEST = """
チーム内で次にこのリクエストを処理するのに最も適したエージェントにタスクを作成してください。 
利用可能な `create_task` 関数を使用してタスクを作成してください。 
リクエスト内容は以下の通りです： 
f"{original_request}"
"""

TEAM_LEADER_TASK_COMPLETENESS_CHECK_INSTRUCTIONS = """
これまでの会話の流れ、特にスレッド内の最新のメッセージを確認してください。 
最終的な結果を向上させる可能性のあるタスクが見つかった場合は、`create_task` 関数を使用してタスクを作成してください。 
タスク作成についてユーザーに確認を求める必要はありません。 
リクエストが完全に処理された場合は、新たなタスクを作成する必要はありません。
"""

TEAM_MEMBER_CAN_DELEGATE_INSTRUCTIONS = """
あなたの名前は '{name}' です。あなたはエージェントチームのメンバーです。 
あなたのチーム名は '{team_name}' です。 
{original_instructions}

- 必要に応じてタスクを委任することができます。委任する場合は、`create_task` 関数を使用し、自分の名前を 'requestor' に指定してください。 
- 割り当てたタスクとその結果を簡潔に報告してください。 
- 他のチームメンバーが専門知識を持っている場合は、助けを求めてください。 
- 自分のタスクが完了したと判断したら、最終的な回答または実行したアクションを報告してください。 
- 以下に、あなたのチームに所属する他のエージェントを示します：{team_description}
"""

TEAM_MEMBER_NO_DELEGATE_INSTRUCTIONS = """
あなたの名前は '{name}' です。あなたはエージェントチームのメンバーです。 
あなたのチーム名は '{team_name}' です。 
{original_instructions}

- あなたはタスクを委任せず、自分に割り当てられたタスクの遂行に集中してください。 
- もし他のエージェントに適したタスクがあると感じた場合は、その旨をレスポンス内で言及してください。ただし、`create_task` を自ら呼び出してはいけません。 
- 自分のタスクが完了したと判断したら、最終的な回答または実行したアクションを報告してください。 
- 以下に、あなたのチームに所属する他のエージェントを示します：{team_description}
"""


## Agent Team Class

In [58]:
# ------------------------------------
# Based on code from Microsoft Corporation.
# Copyright (c) Microsoft Corporation.
#
# Modified by Yuki Matayoshi, 2025.
#
# Licensed under the MIT License.
# ------------------------------------


from typing import Any, Dict, Optional, Set, List
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet, MessageRole, Agent

class _AgentTeamMember:
    """
    Represents an individual agent on a team.

    :param model: The model (e.g. GPT-4) used by this agent.
    :param name: The agent's name.
    :param instructions: The agent's initial instructions or "personality".
    :param toolset: An optional ToolSet with specialized tools for this agent.
    :param can_delegate: Whether this agent has delegation capability (e.g., 'create_task').
                         Defaults to True.
    """

    def __init__(
        self, model: str, name: str, instructions: str, toolset: Optional[ToolSet] = None, can_delegate: bool = True
    ) -> None:
        self.model = model
        self.name = name
        self.instructions = instructions
        self.agent_instance: Optional[Agent] = None
        self.toolset: Optional[ToolSet] = toolset
        self.can_delegate = can_delegate


class _AgentTask:
    """
    Encapsulates a task for an agent to perform.

    :param recipient: The name of the agent who should receive the task.
    :param task_description: The description of the work to be done or question to be answered.
    :param requestor: The name of the agent or user requesting the task.
    """

    def __init__(self, recipient: str, task_description: str, requestor: str) -> None:
        self.recipient = recipient
        self.task_description = task_description
        self.requestor = requestor


class AgentTeam:
    """
    A class that represents a team of agents.

    """

    # Static container to store all instances of AgentTeam
    _teams: Dict[str, "AgentTeam"] = {}

    _project_client: AIProjectClient
    _thread_id: str = ""
    _team_leader: Optional[_AgentTeamMember] = None
    _members: List[_AgentTeamMember] = []
    _tasks: List[_AgentTask] = []
    _team_name: str = ""

    def __init__(self, team_name: str, project_client: AIProjectClient):
        """
        Initialize a new AgentTeam and set it as the singleton instance.
        """
        # Validate that the team_name is a non-empty string
        if not isinstance(team_name, str) or not team_name:
            raise ValueError("Team name must be a non-empty string.")
        # Check for existing team with the same name
        if team_name in AgentTeam._teams:
            raise ValueError(f"A team with the name '{team_name}' already exists.")
        self.team_name = team_name
        if project_client is None:
            raise ValueError("No AIProjectClient provided.")
        self._project_client = project_client
        # Store the instance in the static container
        AgentTeam._teams[team_name] = self

        self.TEAM_LEADER_INSTRUCTIONS = TEAM_LEADER_INSTRUCTIONS
        self.TEAM_LEADER_INITIAL_REQUEST = TEAM_LEADER_INITIAL_REQUEST
        self.TEAM_LEADER_TASK_COMPLETENESS_CHECK_INSTRUCTIONS = TEAM_LEADER_TASK_COMPLETENESS_CHECK_INSTRUCTIONS
        self.TEAM_MEMBER_CAN_DELEGATE_INSTRUCTIONS = TEAM_MEMBER_CAN_DELEGATE_INSTRUCTIONS
        self.TEAM_MEMBER_NO_DELEGATE_INSTRUCTIONS = TEAM_MEMBER_NO_DELEGATE_INSTRUCTIONS
        self.TEAM_LEADER_MODEL = "gpt-4o-mini"


    @staticmethod
    def get_team(team_name: str) -> "AgentTeam":
        """Static method to fetch the AgentTeam instance by name."""
        team = AgentTeam._teams.get(team_name)
        if team is None:
            raise ValueError(f"No team found with the name '{team_name}'.")
        return team

    @staticmethod
    def _remove_team(team_name: str) -> None:
        """Static method to remove an AgentTeam instance by name."""
        if team_name not in AgentTeam._teams:
            raise ValueError(f"No team found with the name '{team_name}'.")
        del AgentTeam._teams[team_name]

    def add_agent(
        self, model: str, name: str, instructions: str, toolset: Optional[ToolSet] = None, can_delegate: bool = True
    ) -> None:
        """
        Add a new agent (team member) to this AgentTeam.

        :param model: The model name (e.g. GPT-4) for the agent.
        :param name: The name of the agent being added.
        :param instructions: The initial instructions/personality for the agent.
        :param toolset: An optional ToolSet to configure specific tools (functions, etc.)
                        for this agent. If None, we'll create a default set.
        :param can_delegate: If True, the agent can delegate tasks (via create_task).
                            If False, the agent does not get 'create_task' in its ToolSet
                            and won't mention delegation in instructions.
        """
        if toolset is None:
            toolset = ToolSet()

        if can_delegate:
            # If agent can delegate, ensure it has 'create_task'
            try:
                function_tool = toolset.get_tool(FunctionTool)
                function_tool.add_functions(agent_team_default_functions)
            except ValueError:
                default_function_tool = FunctionTool(agent_team_default_functions)
                toolset.add(default_function_tool)

        member = _AgentTeamMember(
            model=model,
            name=name,
            instructions=instructions,
            toolset=toolset,
            can_delegate=can_delegate,
        )
        self._members.append(member)

    def _add_task(self, task: _AgentTask) -> None:
        """
        Add a new task to the team's task list.

        :param task: The task to be added.
        """
        self._tasks.append(task)

    def create_team_leader(self, model: str, name: str, instructions: str, toolset: Optional[ToolSet] = None) -> None:
        """
        Add a new agent (team member) to this AgentTeam.

        If team leader has not been created prior to the call to assemble_team,
        then default team leader will be created automatically.

        :param model: The model name (e.g. GPT-4) for the agent.
        :param name: The name of the team leader agent being.
        :param instructions: The initial instructions/personality for the agent.
        :param toolset: An optional ToolSet to configure specific tools (functions, etc.) for the agent.
        """
        assert self._project_client is not None, "project_client must not be None"
        assert self._team_leader is None, "team leader has already been created"
        # List all agents (will be empty at this moment if you haven't added any, or you can append after they're added)
        for member in self._members:
            instructions += f"- {member.name}: {member.instructions}\n"

        self._team_leader = _AgentTeamMember(
            model=model,
            name=name,
            instructions=instructions,
            toolset=toolset,
            can_delegate=True,
        )
        self._team_leader.agent_instance = self._project_client.agents.create_agent(
            model=self._team_leader.model,
            name=self._team_leader.name,
            instructions=self._team_leader.instructions,
            toolset=self._team_leader.toolset,
        )

    def _create_team_leader(self):
        """
        Create and initialize the default 'TeamLeader' agent with awareness of all other agents.
        """
        toolset = ToolSet()
        toolset.add(default_function_tool)
        instructions = self.TEAM_LEADER_INSTRUCTIONS.format(agent_name="TeamLeader", team_name=self.team_name) + "\n"
        self.create_team_leader(
            model=self.TEAM_LEADER_MODEL,
            name="TeamLeader",
            instructions=instructions,
            toolset=toolset,
        )

    def assemble_team(self):
        """
        Create the team leader agent and initialize all member agents with
        their configured or default toolsets.
        """
        assert self._project_client is not None, "project_client must not be None"

        if self._team_leader is None:
            self._create_team_leader()

        for member in self._members:
            if member is self._team_leader:
                continue

            team_description = ""
            for other_member in self._members:
                if other_member != member:
                    team_description += f"- {other_member.name}: {other_member.instructions}\n"

            if member.can_delegate:
                extended_instructions = self.TEAM_MEMBER_CAN_DELEGATE_INSTRUCTIONS.format(
                    name=member.name,
                    team_name=self._team_name,
                    original_instructions=member.instructions,
                    team_description=team_description,
                )
            else:
                extended_instructions = self.TEAM_MEMBER_NO_DELEGATE_INSTRUCTIONS.format(
                    name=member.name,
                    team_name=self._team_name,
                    original_instructions=member.instructions,
                    team_description=team_description,
                )
            member.agent_instance = self._project_client.agents.create_agent(
                model=member.model, name=member.name, instructions=extended_instructions, toolset=member.toolset
            )

    def dismantle_team(self) -> None:
        """
        Delete all agents (including the team leader) from the project client.
        """
        assert self._project_client is not None, "project_client must not be None"

        if self._team_leader and self._team_leader.agent_instance:
            print(f"Deleting team leader agent '{self._team_leader.name}'")
            self._project_client.agents.delete_agent(self._team_leader.agent_instance.id)
        for member in self._members:
            if member is not self._team_leader and member.agent_instance:
                print(f"Deleting agent '{member.name}'")
                self._project_client.agents.delete_agent(member.agent_instance.id)
        AgentTeam._remove_team(self.team_name)

    def process_request(self, request: str, thread_id: str = None) -> None:
        """
        Handle a user's request by creating a team and delegating tasks to
        the team leader. The team leader may generate additional tasks.

        :param request: The user's request or question.
        """
        assert self._project_client is not None, "project client must not be None"
        assert self._team_leader is not None, "team leader must not be None"

        # --- Start of modified Code ---

        # Create a new thread if no thread_id is provided
        print("---")
        if thread_id is None:
            thread = self._project_client.agents.create_thread()
            self._thread_id = thread.id
            print(f"Created thread with ID: {self._thread_id}")
        else:
            self._thread_id = thread_id
            print(f"Set thread with ID: {self._thread_id}")
        
        # --- End of modified Code ---

        team_leader_request = self.TEAM_LEADER_INITIAL_REQUEST.format(original_request=request)
        self._add_task(_AgentTask(self._team_leader.name, team_leader_request, "System"))  # Change the requestor from original "user"

        while self._tasks:
            task = self._tasks.pop(0)
            print("---")
            print(
                f"Starting task for agent '{task.recipient}'. "
                f"Requestor: '{task.requestor}'. "
                f"Task description: '{task.task_description}'."
            )
            
            # --- Start of added Code ---

            # Set the massage metadata
            message_metadata = {"actual_role": task.requestor}

            # Create the massage
            message = self._project_client.agents.create_message(
                thread_id=self._thread_id,
                role="user",
                content=task.task_description,
                metadata=message_metadata # Add the massage metadata
            )

            # --- End of added Code ---

            print(f"Created message with ID: {message.id} for task in thread {self._thread_id}")
            agent = self._get_member_by_name(task.recipient)
            if agent and agent.agent_instance:
                run = self._project_client.agents.create_and_process_run(
                    thread_id=self._thread_id, assistant_id=agent.agent_instance.id
                )
                print(f"Created and processed run for agent '{agent.name}', run ID: {run.id}")
                messages = self._project_client.agents.list_messages(
                    thread_id=self._thread_id
                )

                # --- Start of modified Code ---

                # Find the message for the current run
                for message in messages.data:
                    if message.run_id == run.id:

                        # Set the massage metadata
                        message_metadata  = {"actual_role": task.recipient}

                        # Update the massage for the current run
                        self._project_client.agents.update_message(
                            thread_id=self._thread_id,
                            message_id=message.id,
                            metadata=message_metadata # Add the massage metadata
                        )

                # --- End of modified Code ---
                

            # If no tasks remain AND the recipient is not the TeamLeader,
            # let the TeamLeader see if more delegation is needed.
            if not self._tasks and not task.recipient == "TeamLeader":
                team_leader_request = self.TEAM_LEADER_TASK_COMPLETENESS_CHECK_INSTRUCTIONS
                task = _AgentTask(
                    recipient=self._team_leader.name, task_description=team_leader_request, requestor="System" # Change the requestor from original "user"
                )
                self._add_task(task)

    def _get_member_by_name(self, name) -> Optional[_AgentTeamMember]:
        """
        Retrieve a team member (agent) by name.
        If no member with the specified name is found, returns None.

        :param name: The agent's name within this team.
        """
        if name == "TeamLeader":
            return self._team_leader
        for member in self._members:
            if member.name == name:
                return member
        return None


def _create_task(team_name: str, recipient: str, request: str, requestor: str) -> str:
    """
    Requests another agent in the team to complete a task.

    :param team_name (str): The name of the team.
    :param recipient (str): The name of the agent that is being requested to complete the task.
    :param request (str): A description of the to complete. This can also be a question.
    :param requestor (str): The name of the agent who is requesting the task.
    :return: True if the task was successfully received, False otherwise.
    :rtype: str
    """
    task = _AgentTask(recipient=recipient, task_description=request, requestor=requestor)
    team: Optional[AgentTeam] = None
    try:
        team = AgentTeam.get_team(team_name)
    except:
        pass
    if team is not None:
        team._add_task(task)

        # --- Start of added Code ---

        # Print the task details
        print(f"Added task for agent '{recipient}'. Requestor: '{requestor}'. Task description: {request}.")

        # --- End of added Code ---
        
        return "True"
    return "False"


# Any additional functions that might be used by the agents:
agent_team_default_functions: Set = {
    _create_task,
}

default_function_tool = FunctionTool(functions=agent_team_default_functions)

# 動作確認

## マルチエージェントチームの初期化

In [59]:
# ------------------------------------
# Based on code from Microsoft Corporation.
# Copyright (c) Microsoft Corporation.
#
# Modified by Yuki Matayoshi, 2025.
#
# Licensed under the MIT License.
# ------------------------------------

import os
from typing import Set
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential


# from agent_team import AgentTeam
os.environ["PROJECT_CONNECTION_STRING"] = ""  # Enter the connection string if required

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), 
    conn_str=os.environ["PROJECT_CONNECTION_STRING"] 
)

# Define the user-defined functions
user_function_set_1: Set[Callable[..., Any]] = {
    fetch_current_datetime, 
    fetch_weather
}

user_function_set_2: Set[Callable[..., Any]] = {
    send_email
}

user_function_set_3: Set[Callable[..., Any]] = {
    convert_temperature
}

functions = FunctionTool(functions=user_function_set_1)
toolset1 = ToolSet()
toolset1.add(functions)

# Create an AgentTeam
agent_team = AgentTeam("sample_agent", project_client=project_client)

# Add agents to the team
agent_team.add_agent(
    model="gpt-4o-mini",
    name="TimeWeatherAgent",
    instructions="You are a specialized agent for time and weather queries.",
    toolset=toolset1,
    can_delegate=True,
)

functions = FunctionTool(functions=user_function_set_2)
toolset2 = ToolSet()
toolset2.add(functions)

agent_team.add_agent(
    model="gpt-4o-mini",
    name="SendEmailAgent",
    instructions="You are a specialized agent for sending emails.",
    toolset=toolset2,
    can_delegate=False,
)

functions = FunctionTool(functions=user_function_set_3)
toolset3 = ToolSet()
toolset3.add(functions)

agent_team.add_agent(
    model="gpt-4o-mini",
    name="TemperatureAgent",
    instructions="You are a specialized agent for temperature conversion.",
    toolset=toolset3,
    can_delegate=False,
)

# Initialize the team
agent_team.assemble_team()

## リクエスト

In [60]:
user_request = (
    "こんにちは。現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。"
    "最後に、摂氏を華氏に変換し、結果の概要を記載したメールをサンプル受信者に送信してください。"
)

# Process the user request
agent_team.process_request(
    request=user_request,
    # thread_id=agent_team._thread_id  # Uncomment this line to continue the conversation in the same thread
)

---
Created thread with ID: thread_5oh7t1ZhjcTW75QTWfM3j6Cj
---
Starting task for agent 'TeamLeader'. Requestor: 'System'. Task description: '
チーム内で次にこのリクエストを処理するのに最も適したエージェントにタスクを作成してください。 
利用可能な `create_task` 関数を使用してタスクを作成してください。 
リクエスト内容は以下の通りです： 
f"こんにちは。現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。最後に、摂氏を華氏に変換し、結果の概要を記載したメールをサンプル受信者に送信してください。"
'.
Created message with ID: msg_TJYc4NUX5G7PM0jDc0TIOxYs for task in thread thread_5oh7t1ZhjcTW75QTWfM3j6Cj
Added task for agent 'TimeWeatherAgent'. Requestor: 'TeamLeader'. Task description: 現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。.
Added task for agent 'SendEmailAgent'. Requestor: 'TeamLeader'. Task description: サンプル受信者に摂氏を華氏に変換した結果の概要を記載したメールを送信してください。.
Created and processed run for agent 'TeamLeader', run ID: run_LD60dydtGqGZdIuA7F9jo0WQ
---
Starting task for agent 'TimeWeatherAgent'. Requestor: 'TeamLeader'. Task description: '現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。'.
Created message with I

## 会話履歴の確認

In [61]:
# Fetch and log all messages
messages = project_client.agents.list_messages(
    thread_id=agent_team._thread_id,
)

for data_point in reversed(messages.data):
    last_message_content = data_point.content[-1]
    # Get the actual_role in the message metadata.
    role = data_point.metadata["actual_role"]
    print(f"{role}: {last_message_content.text.value}")
    print("---")

System: 
チーム内で次にこのリクエストを処理するのに最も適したエージェントにタスクを作成してください。 
利用可能な `create_task` 関数を使用してタスクを作成してください。 
リクエスト内容は以下の通りです： 
f"こんにちは。現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。最後に、摂氏を華氏に変換し、結果の概要を記載したメールをサンプル受信者に送信してください。"

---
TeamLeader: リクエストに基づいて、以下のようにタスクを割り当てました：

1. **TimeWeatherAgent**には、"現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。" というタスクを割り当てました。これは、時刻と天気に特化したエージェントがこの情報を迅速に提供できるためです。

2. **SendEmailAgent**には、"サンプル受信者に摂氏を華氏に変換した結果の概要を記載したメールを送信してください。" というタスクを割り当てました。メール送信の専門エージェントであるため、適切な対応が期待できるからです。

両方のタスクは正常に作成されました。これでリクエストは処理されたと判断します。
---
TeamLeader: 現在の時刻を '2023-%m-%d %H:%M:%S' 形式で、また New York の天気を教えてください。
---
TimeWeatherAgent: 現在の時刻は `2023-02-09 22:30:14` です。

また、New York の天気は「晴れ、25°C」となっています。
---
TeamLeader: サンプル受信者に摂氏を華氏に変換した結果の概要を記載したメールを送信してください。
---
SendEmailAgent: サンプル受信者に対して、摂氏を華氏に変換した結果の概要を記載したメールを送信しました。メールは正常に送信されました。
---
System: 
これまでの会話の流れ、特にスレッド内の最新のメッセージを確認してください。 
最終的な結果を向上させる可能性のあるタスクが見つかった場合は、`create_task` 関数を使用してタスクを作成してください。 


## マルチエージェントチームを削除

In [62]:
# Dismantle the team
agent_team.dismantle_team()

Deleting team leader agent 'TeamLeader'
Deleting agent 'TimeWeatherAgent'
Deleting agent 'SendEmailAgent'
Deleting agent 'TemperatureAgent'
